In [1]:
import pandas as pd
from bs4 import BeautifulSoup
from lxml import etree
import unicodedata
import io
import xml.etree.ElementTree as ET
import requests

In [46]:
#50 first records for fiction in ONB after 2009
#SRU: https://obv-at-oenb.alma.exlibrisgroup.com/view/sru/43ACC_ONB?version=1.2&query=alma.national_bibliography_number=OeBC%20and%20alma.local_field_970=56%20and%20alma.%20main_pub_date%3E2009&operation=searchRetrieve
with open('LT_MC_160424_onb_request_50results.xml', encoding='utf8') as f:
    alldata = f.read()

# new_response = BeautifulSoup(alldata) #Parsen der SRU-Textantwort in BeautifulSoup
# gndm = new_response.find_all('ns0:record') #Suchen der einzelnen Records

In [2]:
sru_request = "https://obv-at-oenb.alma.exlibrisgroup.com/view/sru/43ACC_ONB?version=1.2&query=alma.national_bibliography_number=OeBC%20and%20alma.local_field_970=56%20and%20alma.%20main_pub_date%3E2009&operation=searchRetrieve&startRecord={offset}&maximumRecords=50"
offset = 1
alldata = requests.get(sru_request.format(offset=offset))

In [3]:
res_xml = ET.fromstring(alldata.text)

In [4]:
xml_records = []
ns = {'marc': 'http://www.loc.gov/MARC21/slim', 'srw': 'http://www.loc.gov/zing/srw/'}
for record in res_xml.findall(".//marc:record", ns):
    xml_records.append(record)

In [43]:
#Funktion zum Extrahieren der Inhalte:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = record
        
    #Author: 
    creator = xml.findall("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    if creator:
        author = creator[0].text
    else:
        author = "fail"    

        
    gathered = {'Author':lang} # 'ID':idn, 'Language':lang, 'Original Language':oglang}
    return gathered

"""     #IDN:      
    idn = xml.findall("marc:datafield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
                
        
    #Translated Language:      
    lang = xml.findall('.//marc:record/marc:datafield[@tag="041"]/marc:subfield[@code="a"]', namespaces=ns)
    try:
        lang = lang[0].text
    except:
        lang = 'fail'
        
        #Original Language:      
    oglang = xml.findall("marc:datafield[@tag = '041'/marc:subfield[@code = 'h']", namespaces=ns)
    try:
        oglang = oglang[0].text
    except:
        oglang = 'fail'   

 """
    

'     #IDN:      \n    idn = xml.findall("marc:datafield[@tag = \'001\']", namespaces=ns)\n    try:\n        idn = idn[0].text\n    except:\n        idn = \'fail\'\n                \n        \n    #Translated Language:      \n    lang = xml.findall(\'.//marc:record/marc:datafield[@tag="041"]/marc:subfield[@code="a"]\', namespaces=ns)\n    try:\n        lang = lang[0].text\n    except:\n        lang = \'fail\'\n        \n        #Original Language:      \n    oglang = xml.findall("marc:datafield[@tag = \'041\'/marc:subfield[@code = \'h\']", namespaces=ns)\n    try:\n        oglang = oglang[0].text\n    except:\n        oglang = \'fail\'   \n\n '

In [44]:
#Überführen in Tabelle: 
result = [parse_record(record) for record in xml_records]
df_all = pd.DataFrame(result)
df_all

NameError: name 'lang' is not defined

In [35]:
#Show only results where Original Language present: 
df2 = (df_all[df_all['Original Language'] != 'fail'])
df2

KeyError: 'Original Language'

In [ ]:
###ARCHIVE###

In [ ]:
#Funktion: count languages
def howmanylanguages(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
          
    #original language:  
    oglangArray = []    
    oglang = xml.findall("marc:datafield[@tag = '41']/marc:subfield[@code = 'h']", namespaces=ns)
    #print(language)
    if len(language) >= 1:
        for currentLanguage in language:
            languageArray.append(currentLanguage.text)
    else:
        languageArray = 'N/A'
        
    #translated language:   
    lantypeArray = [] 
    langtype = xml.findall("marc:datafield[@tag = '41']/marc:subfield[@code = 'a']", namespaces=ns)
    print(langtype)
    if len(langtype) >= 1:
        for currentLangtype in langtype:
            langtypeArray.append(currentLangtype.text)
    else:
        langtypeArray = 'N/A'
        
        
    numberofolanguages = len(languageArray)
    numberoflangtypes = len(langtypeArray)
    
  
    counting_languages = {'how many original languages':numberofolanguages, 'how many translated languages':numberoflangtypes}
    return counting_languages 

In [ ]:
numbers = [howmanylanguages(item) for item in gndm]
df1 = pd.DataFrame(numbers)
df1

In [ ]:
#display maximum value: 
df1.max()

In [ ]:
#Funktion zum Extrahieren der Inhalte:
def parse_record(record):
    
    ns = {"marc":"http://www.loc.gov/MARC21/slim"}
    xml = etree.fromstring(unicodedata.normalize("NFC", str(record)))
    
    #Author: 
    creator = xml.findall("marc:datafield[@tag = '100']/marc:subfield[@code = 'a']", namespaces=ns)
    if creator:
        author = creator[0].text
    else:
        author = "fail"

    #IDN:      
    idn = xml.findall("marc:controlfield[@tag = '001']", namespaces=ns)
    try:
        idn = idn[0].text
    except:
        idn = 'fail'
        
        
    #Orte und Bezug:      
    place1 = xml.findall("marc:datafield[@tag = '551']/marc:subfield[@code = 'a']", namespaces=ns)
    relation = xml.findall("marc:datafield[@tag = '551']/marc:subfield[@code = 'i']", namespaces=ns)
    
    length = (len(place1))

    if length == 0: 
        found1 = "None"
        found2 = "None"
        found3 = "None"
        found4 = "None"
        found5 = "None"
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = "None"
        rela2 = "None"
        rela3 = "None"
        rela4 = "None"
        rela5 = "None"
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 1: 
        found1 = place1[0].text
        found2 = "None"
        found3 = "None"
        found4 = "None"
        found5 = "None"
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = "None"
        rela3 = "None"
        rela4 = "None"
        rela5 = "None"
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 2: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = "None"
        found4 = "None"
        found5 = "None"
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = "None"
        rela4 = "None"
        rela5 = "None"
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 3: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = "None"
        found5 = "None"
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = "None"
        rela5 = "None"
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 4: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = "None"
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = "None"
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 5: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = place1[4].text
        found6 = "None"
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = relation[4].text
        rela6 = "None"
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 6: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = place1[4].text
        found6 = place1[5].text
        found7 = "None"
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = relation[4].text
        rela6 = relation[5].text
        rela7 = "None"
        rela8 = "None"
        rela9 = "None"
    elif length == 7: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = place1[4].text
        found6 = place1[5].text
        found7 = place1[6].text
        found8 = "None"
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = relation[4].text
        rela6 = relation[5].text
        rela7 = relation[6].text
        rela8 = "None"
        rela9 = "None"
    elif length == 8: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = place1[4].text
        found6 = place1[5].text
        found7 = place1[6].text
        found8 = place1[7].text
        found9 = "None"
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = relation[4].text
        rela6 = relation[5].text
        rela7 = relation[6].text
        rela8 = relation[7].text
        rela9 = "None"
    elif length == 9: 
        found1 = place1[0].text
        found2 = place1[1].text
        found3 = place1[2].text
        found4 = place1[3].text
        found5 = place1[4].text
        found6 = place1[5].text
        found7 = place1[6].text
        found8 = place1[7].text
        found9 = place1[8].text
        rela1 = relation[0].text
        rela2 = relation[1].text
        rela3 = relation[2].text
        rela4 = relation[3].text
        rela5 = relation[4].text
        rela6 = relation[5].text
        rela7 = relation[6].text
        rela8 = relation[7].text
        rela9 = relation[8].text
    elif length >= 10:
        found1 = "Zuviele Treffer"
        found2 = "Zuviele Treffer"
        found3 = "Zuviele Treffer"
        found4 = "Zuviele Treffer"
        found5 = "Zuviele Treffer"
        found6 = "Zuviele Treffer"
        found7 = "Zuviele Treffer"
        found8 = "Zuviele Treffer"
        found9 = "Zuviele Treffer"
        rela1 = "Zuviele Treffer"
        rela2 = "Zuviele Treffer"
        rela3 = "Zuviele Treffer"
        rela4 = "Zuviele Treffer"
        rela5 = "Zuviele Treffer"
        rela6 = "Zuviele Treffer"
        rela7 = "Zuviele Treffer"
        rela8 = "Zuviele Treffer"
        rela9 = "Zuviele Treffer"
    else:
        found1 = "ERROR"  
        found2 = "ERROR"  
        found3 = "ERROR"  
        found4 = "ERROR"  
        found5 = "ERROR"
        found6 = "ERROR"
        found7 = "ERROR"
        found8 = "ERROR"
        found9 = "ERROR"
        rela1 = "ERROR"  
        rela2 = "ERROR"  
        rela3 = "ERROR"  
        rela4 = "ERROR"
        rela5 = "ERROR"
        rela6 = "ERROR"
        rela7 = "ERROR"
        rela8 = "ERROR"
        rela9 = "ERROR"
        
        
    #Zusammenführen:
    gathered = {'Author':author, 'ID':idn, 'Ort1':found1, 'Art1':rela1, 'Ort2':found2, 'Art2':rela2, 
                'Ort3':found3, 'Art3':rela3, 'Ort4':found4, 'Art4':rela4, 'Ort5':found5, 'Art5':rela5,
                'Ort6':found6, 'Art6':rela6, 'Ort7':found7, 'Art7':rela7, 'Ort8':found8, 'Art8':rela8,
                'Ort9':found9, 'Art9':rela9}
    return gathered

In [ ]:
#Überführen in Tabelle: 
result = [parse_record(item) for item in gndm]
df_all = pd.DataFrame(result)
df_all

In [ ]:
#Show only results where Wirkungsort present: 
df2 = (df_all[df_all['Ort1'] != 'None'])
df2

In [ ]:
df2.to_csv("20220128_dnball_authors_places.csv")